# 연관된 데이터 선정

In [7]:
# 프롬프트
from langchain_core.prompts import PromptTemplate

# Prompt
relevance_template = '''
당신은 데이터 과학자입니다. 아래는 연구 데이터 목록입니다.

각 데이터 항목은 다음 컬럼을 가지고 있습니다:
- ID: 각 데이터의 고유키
- 제목
- 설명

[목표]
주어진 연구 주제와 가장 관련성 높은 3~5개의 데이터 항목을 선택하세요.
각 항목에 대해 관련도 점수(0~100)와 선정 사유를 함께 작성하세요.


[Input]
연구 주제: {subject}
연구 설명: {description}

[Data]
데이터 목록:
{data}

[Output]
다음 형식의 JSON을 출력하세요:

{{
  "relevant_id": ["id1", "id2", "id3"],
  "relevance": [95, 87, 80],
  "reason": [
    "id1이 높은 관련도를 갖는 이유를 구체적으로 기술",
    "id2가 관련 있다고 판단한 근거를 기술",
    "id3을 선정한 이유를 기술"
  ]
}}
'''

relevance_prompt = PromptTemplate.from_template(relevance_template)


In [8]:
# Schema
from typing_extensions import Annotated
from pydantic import Field, BaseModel, conint

# Schema
class IDRelevance(BaseModel):
    relevant_id: Annotated[
        list[str],
        Field(
            ..., 
            max_items=5, 
            min_items=3,
            description=(
                "가장 관련성이 높은 데이터의 ID 목록, 길이 최소 3개/최대 5개"
                "각 ID에 대해 relevance(관련도 점수)와 reason(선정 사유)을 동일한 인덱스 위치에 작성합니다. "
            ), 
        )
    ]
    relevance: Annotated[
        list[
            Annotated[
                int, 
                Field(ge=0, le=100, description="0~100 정수")
            ]
        ],
        Field(
            ..., 
            max_items=5, 
            min_items=3,
            description="각 ID에 대응하는 관련도 점수 (0~100 정수)",
        )
    ]
    reason: Annotated[
        list[str],
        Field(
            ..., 
            max_items=5, 
            min_items=3,
            description="각 ID가 선정된 이유를 설명하는 문자열 목록. relevant_id와 인덱스가 일치해야 합니다.",
        )
    ]

In [9]:
from langchain_openai import ChatOpenAI
import pandas as pd
import json


df = pd.read_csv('../data/search_results.csv', encoding='UTF-8', low_memory=False)

with open("../data/input_data.json", "r", encoding="utf-8") as f:
    input_data = json.load(f)
    
subject, description = input_data['dataset_title_etc_main'], input_data['dataset_expl_etc_main']


prompt = relevance_prompt.invoke(
    {
        'subject': subject, 
        'description': description,
        'data': df[['svc_id', 'dataset_title_etc_main', 'dataset_expl_etc_main']].to_dict(orient="records"),
    }
)

# sLLM
sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

structured_sllm = sllm.with_structured_output(IDRelevance)
res = structured_sllm.invoke(prompt)

print(res)

relevant_id=['b37f0c9413eeb7c45f6fe31cbe3a41ef', 'a83eb22a4a62949d83c70662acdf439a', '90ca58cfe0896194588fe73299e585f1', '4f8e04de8044dd201965353514748c13', 'd9ffc0252df265727ba8a3e446bddefb'] relevance=[95, 87, 80, 75, 70] reason=["'Architectural Urbanism: Melbourne/Seoul - KTA projects'는 연구 주제와 직접적으로 관련이 있으며, KTA의 프로젝트들이 전시된 내용을 포함하고 있어 가장 높은 관련도를 가집니다.", "'Urban Architectures'는 서울과 멜버른의 건축적 접근 방식을 탐구하는 전시의 일환으로 선정된 프로젝트로, 연구 주제와의 연관성이 높습니다.", "'Grace Park House'는 멜버른의 건축적 맥락에서 기존 건물과의 관계를 탐구하며, 연구 주제의 '도시 내에서의 건축' 개념과 잘 맞아 관련성이 있습니다.", "'Urban Box'는 멜버른의 인필 하우징을 다루며, 도시 환경과의 관계를 고려한 설계로 연구 주제와의 연관성이 있습니다.", "'Monash University Logan Hall'는 커뮤니티와의 상호작용을 강조하며, 도시 내에서의 건축적 역할을 탐구하는 점에서 연구 주제와 관련이 있습니다."]
